In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation,BatchNormalization,Input,Embedding,Dot,Dense,Flatten
from tensorflow.keras.callbacks import ModelCheckpoint,LearningRateScheduler,TensorBoard,EarlyStopping

from wordcloud import WordCloud
%matplotlib inline

In [2]:
import os

In [3]:
INPUT_DIR = os.path.join("..","artifacts","raw")

In [4]:
rating_df = pd.read_csv(INPUT_DIR+"/animelist.csv" , low_memory=True,usecols=["user_id","anime_id","rating"])

In [5]:
rating_df.head()

,user_id,anime_id,rating
0,0,67,9
1,0,6702,7
2,0,242,10
3,0,4898,0
4,0,21,10


In [6]:
len(rating_df)

109224747

#### DATA PROCESSING

In [7]:
n_ratings = rating_df['user_id'].value_counts()
rating_df = rating_df[rating_df['user_id'].isin(n_ratings[n_ratings>=400].index)].copy()

In [8]:
len(rating_df)

71418114

In [9]:
min_rating =min(rating_df["rating"])
min_rating

0

In [10]:
max_rating =max(rating_df["rating"])
max_rating

10

In [11]:
avg_rating = np.mean(rating_df['rating'])
avg_rating

4.047793589172629

In [12]:
rating_df['rating'] = rating_df['rating'].apply(lambda x : (x-min_rating)/(max_rating-min_rating)).values.astype(np.float64)

In [13]:
rating_df.duplicated().sum()

1

In [14]:
rating_df.isnull().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [15]:
user_ids = rating_df["user_id"].unique().tolist()
user2user_encoded = {x : i for i , x in enumerate(user_ids)}
user2user_decoded = {i : x for i , x in enumerate(user_ids)}
rating_df["user"] = rating_df["user_id"].map(user2user_encoded)


In [16]:
n_users = len(user2user_encoded)

In [17]:
n_users

91641

In [18]:
anime_ids = rating_df["anime_id"].unique().tolist()
anime2anime_encoded = {x : i for i , x in enumerate(anime_ids)}
anime2anime_decoded = {i : x for i , x in enumerate(anime_ids)}
rating_df["anime"] = rating_df["anime_id"].map(anime2anime_encoded)

In [19]:
n_anime = len(user2user_encoded)

In [20]:
n_anime

91641

In [21]:
rating_df = rating_df.sample(frac=1,random_state=43).reset_index(drop=True)

In [22]:
rating_df.head(2)

,user_id,anime_id,rating,user,anime
0,191596,27787,0.0,49525,1955
1,32036,4187,0.0,8217,6628


In [23]:
X = rating_df[["user","anime"]].values
y = rating_df["rating"]


In [24]:
test_size = 1000
train_indices = rating_df.shape[0] - test_size